In [1]:
from openpyxl import Workbook
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font, Alignment, Border, Side, PatternFill
import pandas as pd
import string
import numpy

# Creando el libro de excel
wb = Workbook()
ws = wb.create_sheet('Reporte', index = 0)

#Propiedades
ws.sheet_view.showGridLines = False
ws.sheet_view.zoomScale = 70

for column in ["A","B","C","D","E","F","G","H","I","J","K","L","M","N"]:
    ws.column_dimensions[column].width = 17

data = {
    "Product Name": ["ID", "Product 0", "Product 1", "Product 2", "Product 3", 
                    "Product 4","Product 5", "Product 6", "Product 7", 
                    "Product 8", "Product 9", "Product 10"],
    "Trazado": [30, 30, 30, 30, 20, 20, 20, 20, 20, 10, 10, 10],
    "Sales Month 2": [30, 30, 25, 25, 20, 19, 15, 15, 15, 10, 5, 5],
    "Sales Month 3": [30,30, 5, 10, 20, 10, 30, 30, 5, 5, 35, 10],
    "Sales Month 4": [30, 30, 5, 5, 35, 10, 30, 30, 5, 5, 35, 10],
    "Calificación": ['0', '0', '1', '1', '1', '1', '1', '1', '2', '2', '2', '2'],
    "Sales Month 6": [ 20 , 30, 5, 5, 35, 10, 30, 30, 5, 5, 35, 10],
}

df = pd.DataFrame(data)
df = df.set_index(df.columns[0])

"""introduciendo los datos"""
for row in dataframe_to_rows(df, index = False, header=True):
    ws.append(row)

"""introducir columnas"""
columnas_1 = [i for i in string.ascii_uppercase] #letras de columnas
columnas_2 = [f'{i}{j}' for j in string.ascii_uppercase for i in string.ascii_uppercase] #letras de columnas
columnas = columnas_1 + columnas_2


columna = 4
ws.insert_cols(columna)
ws[f'{columnas[columna-1]}{ws.min_row}'].value = 'Total'


filas = numpy.arange(2,len(df)+1)
for row in ws.iter_rows(min_row=ws.min_row+1, max_row=ws.max_row, min_col=columna, max_col=columna):
    for cell in row:
        cell.value = f'= B{cell.row} + C{cell.row}'

########################
columna = 8
ws.insert_cols(columna)
ws[f'{columnas[columna-1]}{ws.min_row}'].value = 'Total'


filas = numpy.arange(2,len(df)+1)
for row in ws.iter_rows(min_row=ws.min_row+1, max_row=ws.max_row, min_col=columna, max_col=columna):
    for cell in row:
        cell.value = f'= D{cell.row} + E{cell.row} + F{cell.row} + G{cell.row}'

"""Estilos"""
#Fuente
fuente_letra_azul_oscuro = Font(bold= True, size='11',color= '00003366')

#Alineamiento
alinear_centro = Alignment(horizontal = 'center', vertical = 'center')

#borde
borde_4_lados = Border(
                        left=Side(style='thin'),  
                        right=Side(style='thin'), 
                        top=Side(style='thin'), 
                        bottom=Side(style='thin')
                        )
#relleno
relleno_azul_categoria = PatternFill(
                        start_color= '4787B7',
                        end_color= '4787B7',
                        fill_type= 'solid'
                        )
#encabezado                  
for row in ws.iter_rows(min_row=ws.min_row, max_row=1, min_col=ws.min_column, max_col=ws.max_column):
    for cell in row:
        cell.font = fuente_letra_azul_oscuro 
        cell.fill = relleno_azul_categoria      
        cell.alignment = alinear_centro
        cell.border = borde_4_lados

#cuerpo
for row in ws.iter_rows(min_row=ws.min_row+1, max_row=ws.max_row, min_col=ws.min_column, max_col=ws.max_column):
    for cell in row:
        # cell.font = fuente_letra_azul_oscuro 
        # cell.fill = relleno_azul_categoria      
        # cell.alignment = alinear_centro
        cell.border = borde_4_lados


In [ ]:
"""merge cell with column trazados"""
table = []
for row in ws.iter_rows(min_row=ws.min_row, max_row=ws.max_row, min_col=ws.min_column, max_col=ws.max_column):
    for cell in row:
       table.append(cell)
table = numpy.array(table).reshape(ws.max_row, ws.max_column)

columna_A = [cell.value for cell in table[1:,0]]


conjunto = list(set(columna_A))[::-1]

number_rows = [columna_A.count(i) for i in conjunto]

cumsum_inverso = numpy.cumsum(number_rows[::-1])
cumsum = numpy.cumsum(number_rows)

cumsum = numpy.insert(cumsum, 0, 1)
print(cumsum_inverso, cumsum[0:-1])

for i,j in zip(cumsum_inverso, cumsum[0:-1]):
    min_row = len(ws['A']) - i + 1
    max_row = len(ws['A']) - j + 1
    ws.merge_cells(f'A{min_row}:A{max_row}')
    print(min_row, max_row)

[ 3  8 12] [1 4 9]
11 13
6 10
2 5


In [ ]:
"""merge cell with column segunda"""
table = []
for row in ws.iter_rows(min_row=ws.min_row, max_row=ws.max_row, min_col=ws.min_column, max_col=ws.max_column):
    for cell in row:
       table.append(cell)
table = numpy.array(table).reshape(ws.max_row, ws.max_column)

columna_B = [cell.value for cell in table[1:,1]]
conjunto = list(set(columna_B))[::-1]

number_rows = [columna_B.count(i) for i in conjunto]


cumsum_inverso = numpy.cumsum(number_rows[::-1])
cumsum = numpy.cumsum(number_rows)

cumsum_add = numpy.insert(cumsum, 0, 0)
print(cumsum_add)



for i,j in zip(cumsum_inverso, cumsum_add[0:-1]):
    # min_row = len(ws['B']) - i + 1
    max_row = len(ws['B']) - j + 1
    # ws.merge_cells(f'B{min_row}:B{max_row}')
    print(max_row)

#min_row [ 2  3  6  7  8 10 12]
#max_row [3,5,6,7,10, 11, 13]

[ 0  2  4  5  6  9 10 12]
14
12
10
9
8
5
4


In [ ]:
"""separacion de trazados"""
table = []
for row in ws.iter_rows(min_row=ws.min_row, max_row=ws.max_row, min_col=ws.min_column, max_col=ws.max_column):
    for cell in row:
       table.append(cell)
table = numpy.array(table).reshape(ws.max_row, ws.max_column)

columna = [cell.value for cell in table[1:,0]]


conjunto = list(set(columna))[::-1]

number_rows = [columna.count(i) for i in conjunto]
cumsum = numpy.cumsum(number_rows)

for i in cumsum[::-1]:
    row = i + 2
    ws.insert_rows(idx=row, amount=3)

In [ ]:
#  ws.merge_cells('B1:C2')



# from copy import copy
# for cell in ws[1]:
#     for cell_ in ws[20]:
#         cell_.value = copy(cell.value)
        


In [ ]:
wb.save('Prueba_2.xlsx')

In [ ]:
"""separacion de trazados"""
table = []
for row in ws.iter_rows(min_row=ws.min_row, max_row=ws.max_row, min_col=ws.min_column, max_col=ws.max_column):
    for cell in row:
       table.append(cell)
table = numpy.array(table).reshape(ws.max_row, ws.max_column)

columna = [cell.value for cell in table[1:,0]]


conjunto = list(set(columna))[::-1]

number_rows = [columna.count(i) for i in conjunto]
cumsum = numpy.cumsum(number_rows)

for i in cumsum[::-1]:
    row = i + 2
    ws.insert_rows(idx=row, amount=3)



[12  9  4]
